As the train data is complete, and interactions of users are arranged chronologically. Unlike totally shuffle, I thought it would be one way to split train/valid data userwise as follows:

In [1]:
import numpy as np
import pandas as pd

Load train.csv

In [2]:
train_dtypes_dict = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "int8",
    #"task_container_id": "int16",
    #"user_answer": "int8",
    "answered_correctly": "int8",
    #"prior_question_elapsed_time": "float32", 
    #"prior_question_had_explanation": "boolean"
}

data = pd.read_csv("../input/riiid-test-answer-prediction/train.csv",
                         nrows=10**5, #how many rows to read
                         usecols = train_dtypes_dict.keys(),
                         dtype=train_dtypes_dict,
                        )

Choose question type interaction only if you want.

In [3]:
data_q = data[data.content_type_id == 0]
data_q.head()

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
0,0,0,115,5692,0,1
1,1,56943,115,5716,0,1
2,2,118363,115,128,0,1
3,3,131167,115,7860,0,1
4,4,137965,115,7922,0,1


Get users and their interaction counts

In [4]:
users_i_counts = data_q.user_id.value_counts()
user_list = users_i_counts.keys()
print(user_list[:5])
print(users_i_counts.head())

Int64Index([1283420, 24418, 1440282, 1282581, 2041025], dtype='int64')
1283420    7476
24418      6283
1440282    5620
1282581    4993
2041025    4343
Name: user_id, dtype: int64


Choose how many users you want to train depending on their interaction counts. For example, if you want interaction counts greater than 6000, you can choose the first two users in user_list. And valid_num below controls how many valid data you want.

In [5]:
user_num = 2 #control how many user to train
valid_num = 2 #control how many valid data to keep
train_data = pd.concat([data_q[data_q.user_id==u][:-valid_num] for u in user_list[:user_num]], axis=0)
valid_data = pd.concat([data_q[data_q.user_id==u][-valid_num:] for u in user_list[:user_num]], axis=0)

In [6]:
train_data

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
54058,54058,0,1283420,3822,0,1
54059,54059,29795,1283420,638,0,1
54060,54060,91982,1283420,4231,0,0
54061,54061,117342,1283420,7978,0,0
54062,54062,145733,1283420,8225,0,1
...,...,...,...,...,...,...
8331,8331,14243544102,24418,8895,0,0
8332,8332,14243573690,24418,5016,0,1
8333,8333,14243646189,24418,4673,0,0
8334,8334,14243678701,24418,4174,0,1


In [7]:
valid_data

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
61625,61625,22373215518,1283420,3400,0,0
61626,61626,22373215518,1283420,3399,0,1
8336,8336,14243703953,24418,4612,0,1
8337,8337,14243735782,24418,4163,0,1


Check where different users separate in train_data

In [8]:
train_data[7472:7476]

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
61623,61623,22364257029,1283420,2039,0,1
61624,61624,22373215518,1283420,3401,0,1
1874,1874,0,24418,7900,0,0
1875,1875,24224,24418,7876,0,0


So as you can see, valid data always come after train data. It's a time-aware userwise train-valid-split method.